<a href="https://colab.research.google.com/github/faranak5171/Stable-Diffusion/blob/main/Latent_DiffusionModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adapted from : https://github.com/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb

In [7]:
!pip install -q diffusers==0.8.0
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"

     |████████████████████████████████| 433 kB 32.7 MB/s 


In [9]:
import os
from PIL import Image,ImageDraw
import cv2
import numpy as np
from IPython.display import HTML
from base64 import b64encode

import torch
from torch import autocast
from torch.nn import functional as F
from diffusers import StableDiffusionPipeline, AutoencoderKL
from diffusers import UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler
from diffusers.schedulers.scheduling_ddim import DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from google.colab import output


In [10]:
device = 'cuda'

# Enable a Jupyter widget manager
output.enable_custom_widget_manager()
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [11]:
# Load stable diffusion model to GPU
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, revision='fp16', torch_dtype=torch.float16, use_auth_token=True)
pipe = pipe.to(device)

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
prompt = 'cute shiba inu dog'
